# Risk Framework

Generally, we can categorize between systematic and unsystematic risk. Systematic risk generally refers to events affecting the overall market, whereas unsystematic risk refers to our implemented strategy.

### Systematic Risk

When it comes to systematic risk, or market risk, we cannot reduce this risk by diversifying our portfolio. Given that our strategy is market-neutral, the risk is somewhat reduced as in the case of the overall market breakdown, we would still be hedged on the downside. 

One way to measure said risk is by leveraging the Value at Risk (VaR) statistic, which calculates the maximum potential loss a portfolio could experience within a given time frame.


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm

df = pd.read_csv("returns.csv")
df_daily = df.groupby(pd.to_datetime(df["date"]).dt.floor("d"))["total_post_fees"].sum()
# 100,000 USD portfolio
value = 100000
mu = np.mean(df_daily)
sigma = np.std(df_daily)
alpha = norm.ppf(1-0.99, mu, sigma)

print(f"The VaR on a daily basis for our strategy is ${value-value*(alpha+1):2.2f} with a 99% confidence")

The VaR on a daily basis for our strategy is $1191.96 with a 99% confidence



### Unsystematic Risk

Unsystematic risk refers to the risks directly associated with the sub-sector or assets at hand. In our case, these would be in direct relationship to our pair-trading strategy.

#### Co-Integration/Correlation Breakdown

The data in our sample is very limited and it is therefore difficult to assess the reliability of our measures. One possible way to mitigate this risk is by monitoring the spread and exit the positions when our z-score crosses 2. Similarly, one could have a larger "bucket" of assets to short/long where it is less likely that the correlation would break as quickly/easily.

## Live Monitoring (Risk Engine)

The following part describes how a real time risk engine could be implemented for a given strategy.

## Metrics

#### VaR

As described above, VaR can be calculated taking the returns into account. One can make the aggregation on various time periods and consistently update this number as new data comes in (when period cut-offs happen).

#### MMR

Depending on the exchange (in our case Hyperliquid), one would need to consistently monitor the margin requirements. This can become somewhat complex if we have many sub-accounts. In the case of Hyperliquid, the maintenance margin is between 1% and 16.7%. Therefore, if any of the sub-accounts come close, we should be alerted.

| Description      | Limit |
| ----------- | ----------- |
| MMR within 40% of liquidation     | Alert level 3 (informational)  |
| MMR within 30% of liquidation  | Alert Level 2 (medium)       |
| MMR within 20% of liquidation   | Alert level 1 (critical) |

#### Risk Limits

Ideally, we should also stay within limits as defined via our risk framework. One could be as detailed as needed, e.g., having DeFi vs CeFi limits, counterparty limits as well as limits for specific assets. Given that we are currently focusing on DeFi, we will not take CeFi limits into consideration. Instead, we can focus on couterparty limits, asset exposure limits as well as strategy-specific limits, e.g., trend-following, mean-reversion, or, as we have here, pair-trading.

* Counterparty Limit: There will be a counterparty limit for the exchange that we are trading on. It is imperative to spread assets across exchanges where possible so that in case of unforeseen events, one would not lose the whole portfolio value. 

* Strategy Limit: Generally speaking, since we are always long an asset and short another one at the same time, the limits for that specific strategy could be higher when comparing to others where we do not have a direct hedge. 

* Asset Limit: We should also have limits for the respective assets after they have undergone a review. Since we are not always short a particular asset as we are either long/short the spread, there will be instances where we are potentially longing a "bad" asset. In our example this is not the case but there are instances where we could set up a pair trade with a currency that is for example heavily controlled/could be manipulated as it has happened over the summer (TRB for example). While one cannot completely eliminate this risk, one can at least review the assets at hand and also monitor when large on-chain deposits onto exchanges happen. This would mitigate the risk of potential funding rate fluctuations/correlation breakdowns in the case of manipulations.

For our case, we could have something along these limits and alert:

| Description      | Limit |
| ----------- | ----------- |
| Limit Strat (tag: pair-trade)      | 30%       |
| Limit Asset (SOL)   | 20%        |
| Limit Asset (BTC)   | 20%        |
| Limit Counterparty (Hyperliquid)   | 20%        |


